In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import pandas as pd
import os

#  %matplotlib inline  



First functions: look at statistics file, and get a couple of info on the velocity, temperature, etc. 
Second set of functions: read the parameter file and extract everything from it in a dictionnary. (Be careful, as outputs are strings, not numbers.)


Important points to check: is the system stable enough? Is it in steady state? Are the velocity reasonnable?

In [2]:
def open_statistics(fname):
    """ Open the statistics file output by CIG-ASPECT
    
    return a pandas table, where names are taken from the statistics file.
    """
    # header:
    header = []
    header_read = True

    with open(fname) as f:
        while header_read :
            line = f.readline()
            if line[0] == '#':
                header.append(line[2:-1])
            else:
                header_read = False
                
    # data
    values = pd.read_csv(fname, skiprows=len(header), header=None, delim_whitespace=True, names=header)
    return values



def plot_statistics(fname, output={}):
    """ Plot the required fields from statistics file output by CIG-ASPECT 
    
    By default, output is (as function of time step)
     - first subfigure with: '12: RMS velocity (m/s)', '13: Max. velocity (m/s)'
     - second with '14: Minimal temperature (K)', '15: Average temperature (K)', '16: Maximal temperature (K)'
     - third with '2: Time (seconds)'
     
     Any provided output will be added as an additional subfigure (all outputs on same subfigure)
    """
    
    values = open_statistics(fname)
    
    if len(output) == 0:
        fig, ax = plt.subplots(3,1,  figsize=[10,10], sharex = True)
    else:
        fig, ax = plt.subplots(4,1,  figsize=[12,10], sharex = True)
        
        
    Velocities = ["RMS velocity (m/s)", "Max. velocity (m/s)"]
    Temperatures = ["Minimal temperature (K)", "Average temperature (K)", "Maximal temperature (K)"]
    Time = "Time (seconds)"
    
    header = values.columns.values
    

    for name in header: 
        
        for v in Velocities:
            if name[-len(v):] == v:
                values.plot(x=header[0], y=name, ax=ax[0])
        for t in Temperatures:
            if name[-len(t):] == t:
                values.plot(x=header[0], y=name, ax=ax[1])
        if name[-len(Time):] == Time:
            values.plot(x=header[0], y=name, ax=ax[2])
    
    return fig, ax



In [3]:
#mypath = 'Regim_Diagram_2D'
mypath = 'Feb28'

for (dirpath, dirnames, filenames) in os.walk(mypath):
    for file in filenames:
        if file == 'statistics':
            print(dirpath)
            fig, ax = plot_statistics(dirpath+'/'+file)
            plt.savefig(dirpath+'/statistics.pdf')
            plt.close(fig)
    

Feb28/P1e4_Ra1e4/output-noPT
Feb28/P1e4_Ra1e4/output-PT_+dTdP
Feb28/P1e4_Ra1e4/output-PT_-dTdP
Feb28/P1e4_Ra1e4/output-PT_2
Feb28/P1e4_Ra1e4/output-PT_3
Feb28/P1e4_Ra1e5/output-+PT
Feb28/P1e4_Ra1e5/output--PT
Feb28/P1e4_Ra1e5/output-noPT


In [65]:
file = 'Feb28/P1e4_Ra1e5/output-noPT/parameters.prm'


def search(objs, b):
    return objs.index(b)
   


def set_param(line):
    
    debut = line.index("=")
    param_name = line[1:debut]
    param_name = ' '.join(word for word in param_name)
        
    param_value = line[debut+1:]
    
    if '#' in param_value: 
        fin = param_value.index("#")
        param_value = param_value[:fin]

    return param_name, param_value
    

    
    
def parameter_file(file):
    """ Read parameter file, and return a dictionnary with all values. """
    
    with open(file) as f:
        file_param = f.read().splitlines()

    parameters = dict({}) 
    section = ""
    for line in file_param:

        line = line.split()
        if len(line)==0:
            pass
        elif line[0] == "#":
            pass
        elif line[0] == "end":
            if ":" in section:
                section = section[:section.rindex(":")]
            else:    
                section = ""
        elif line[0] == "subsection":
            name_section = ' '.join(word for word in line[1:])
            if section == "":
                section = name_section
            else:
                section = section+": "+name_section
            #print(section)

        elif line[0] == 'set':

            name, value = set_param(line)
            if section == "":
                parameters[name] = value
            else:
                if section in parameters:
                    parameters[section][name] = value
                else:
                    parameters[section] = dict({})
                    parameters[section][name] = value
    return parameters



{'Phase transition width': ['0.05'], 'Phase transition temperature': ['0.3'], 'Compute quadratic pressure profile from gravity': ['true'], 'Phase transition Clapeyron slope': ['-1e4'], 'Phase transition density change': ['0.0'], 'Phase transition radius': ['0.0']} {'Function constants': [], 'Variable names': ['x,y'], 'Function expression': ['1e4']}
{'Bottom temperature': ['0'], 'Left temperature': ['1'], 'Right temperature': ['0'], 'Top temperature': ['0']}
